<a href="https://colab.research.google.com/github/pedrobcavalcante/Backtest-trading-strategies/blob/main/Gerador_de_Par%C3%A2metros_para_estrat%C3%A9gias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [11]:
import backtesting 
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import os.path
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
backtesting.set_bokeh_output(notebook=False)


In [12]:
from backtesting.test import SMA, GOOG
# GOOG.to_csv("data/GOOG.csv")


In [13]:
def fixData(data):
    columns = ['open_time', 'Open', 'High', 'Low', 'Close', 'Volume',
               'close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore']
    df = pd.DataFrame(data, columns=columns)
    df.index = [dt.datetime.fromtimestamp(x/1000.0).date() for x in df.close_time]
    df.sort_values(["open_time"], ascending=False)
    df = df.astype('double')
    df['Volume'] = df['Volume'].astype('int')
    df = df.drop(['qav', 'num_trades', 'taker_base_vol',
                 'taker_quote_vol', 'ignore'], axis=1)
    return df


In [14]:
def get_bars(symbol, interval='1h'):
    root_url = 'https://api.binance.com/api/v1/klines'
    filename = "data/"+symbol+"_"+interval+".csv"
    url = root_url + '?symbol=' + symbol + '&interval=' + interval+"&limit=1001"
    print(url)
    data = json.loads(requests.get(url).text)
    df = fixData(data)
#    df.to_csv(filename, sep=';')
    return df


In [15]:
dataframe = get_bars("BTCUSDT", "1d")
dataframe


https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1d&limit=1001


In [ ]:
class SmaCross(Strategy):
    n1 = 0
    n2 = 0

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

    def setInitialParams(self, valor1, valor2):
        self.n1 = valor1
        self.n2 = valor2


In [ ]:
def gerarParametros(dataframe, name, myStrategy, tamanho):
    maiorRetorno = {}
    maoirAcerto = {}
    BoasEstratégiasParaSalvar = []
    for s1 in range(tamanho):
        s1 += 1
        print("\ntestando nos valores ", s1, end=" ")
        for s2 in range(tamanho):
            s2 += 1
            print(",", s2, end="")
            try:
                SmaCross.setInitialParams(SmaCross, s1, s2)
                bt = Backtest(dataframe, myStrategy,
                              cash=100000000, exclusive_orders=True)
                output = bt.run()
                if maiorRetorno == {}:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if maoirAcerto == {}:
                    maoirAcerto = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > 0 and output['# Trades'] > 9:
                    BoasEstratégiasParaSalvar.append(
                        {"sma1": s1, "sma2": s2, "output": output})
        #         if output['Win Rate [%]'] > maiorRetorno["output"]['Win Rate [%]'] and output['# Trades'] > 9:
        #             maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
            except Exception as e:
                print(e)
    if not os.path.exists('estrategy results'):
        os.makedirs('estrategy results')
    bestParamtes = pd.DataFrame(BoasEstratégiasParaSalvar)
    bestParamtes.to_csv("estrategy results/"+name+".csv", sep=';')


In [ ]:
data = json.loads(requests.get(
    "https://api.binance.com/api/v1/exchangeInfo").text)
data = data['symbols']


In [ ]:
pares = []
matches = [x for x in data if x['quoteAsset'] == "USDT"]
for matche in matches:
    if  not matche['symbol'] == "BTCUSDT" and not matche['symbol'] == "ETHUSDT":        
        print('\n', matche['symbol'])
        dataframe = get_bars(matche['symbol'], "1w")
        filename = matche['symbol']+"_"+"1w"
        gerarParametros(dataframe, filename, SmaCross, 100)



 BNBUSDT
https://api.binance.com/api/v1/klines?symbol=BNBUSDT&interval=1w&limit=1001


KeyboardInterrupt: 

In [ ]:
# print(maiorRetorno)
# print(maoirAcerto)
# print(BoasEstratégiasParaSalvar)
# gerarParametros()
# bestParamtes = pd.DataFrame (BoasEstratégiasParaSalvar)
# bestParamtes.to_csv("estrategy results/melhores parametros.csv", sep=';')


# Nova seção


In [ ]:

# SmaCross.setInitialParams(SmaCross,12, 15)
# bt = Backtest(GOOG, SmaCross,cash=1000,exclusive_orders=True)
# output = bt.run()
# print(output)
# SmaCross.setInitialParams(SmaCross,29, 30)
# bt = Backtest(dataframe, SmaCross,cash=1000000)
# output = bt.run()
# output
# bt.plot()
